#### Import libaries

In [ ]:
%pip install scipy
%pip install matplotlib
%pip install ipywidgets
%pip install numpy

In [2]:

# circuit_solution=run_sim(V0, C, L, R_total, 1)
# Perform the cumulative integration
# magnetic_field = mu_0 * N_turns * np.abs(circuit_solution.y[1]) / L
# velocity = cumulative_trapezoid(circuit_solution.y[2], circuit_solution.t)
# position = cumulative_trapezoid(velocity, circuit_solution.t[:-1])
# plot_velocity_and_magnetism(circuit_solution.t[:999999], velocity[0:999999], magnetic_field[:999999])

# print(circuit_solution)

In [3]:
# Define the natural frequency and critical damping resistance
# R_intrinsic = coil_wire_resistance + capacitor_ESR
# omega_0 = 1 / np.sqrt(L * C)
# R_critical = 2 * np.sqrt(L / C)
# zeta = R_intrinsic / (2 * np.sqrt(L / C))

# Print info about frequency
# print(f"Natural Frequency (ω0): {omega_0:.3e} rad/s")
# print(f"R_intrinsic: {R_intrinsic:.3e} Ω")
# print(f"Critical Damping Resistance: {R_critical:.3e} Ω")
# print(f"R_critical - R_intrinsic: {R_critical - R_intrinsic} (close to zero is critically damped)")
# print(f"Damping Ratio (ζ): {zeta:.3e}")

# Damping resistor
# R_damp = R_critical - R_intrinsic
# if (R_damp > 0):
#     print("Adding a damping resistor of {:.3e} Ω to the simulation".format(R_damp))
#     R_total = R_damp + R_intrinsic
# else:
#     R_total = R_intrinsic
# print(f"Total resistance: {R_total:.3e} Ω" )


In [4]:
# Calculate magnitude of magnetic field
# V, I = circuit_solution.y
# B = mu_0 * (N_turns / coil_length) * I
# B_max = max(B, key=abs)
# print(f"B_max: {B_max:.3} Tesla")

# Create a third y-axis for the magnetic field strength
# fig, ax3 = plt.subplots(figsize=(10, 5))
# ax3.plot(t, B, 'g-', label='Magnetic Field (T)')
# ax3.set_xlabel('Time (µs)')  # Set x-axis label to microseconds
# ax3.set_ylabel('Magnetic Field (T)', color='g')
# ax3.tick_params(axis='y', labelcolor='g')

# Plot the magnetic flux
# fig, ax4 = plt.subplots(figsize=(10, 5))
# ax4.plot(t, Phi, 'm-', label='Magnetic Flux (Wb)')
# ax4.set_xlabel('Time (µs)')  # Set x-axis label to microseconds
# ax4.set_ylabel('Magnetic Flux (Wb)', color='m')
# ax4.tick_params(axis='y', labelcolor='m')

In [5]:


# plot_solution(simulate_circuit(V0, C, L, R_total, 1))

# def plot_simulation(V0, C, L, R, duration_seconds) :
#     plot_solution(simulate_circuit(V0, C, L, R, duration_seconds))

# plot_simulation(V0, C, L, R_total, 1) 

# interact(plot_simulation, V0=V0, C=cap_slider, L=L, R=R_total, duration_seconds=1)
              

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, FloatLogSlider, IntSlider
from scipy.integrate import solve_ivp, cumulative_trapezoid
import numpy as np
from collections import namedtuple
import plots

mu_0 = 4 * np.pi * 1e-7  # Permeability of free space in T·m/A

# Projectile properties
proj_diameter = 0.005  # Diameter in meters
proj_length = 0.010  # Length in meters
proj_density = 7870  # Density of iron in kg/m^3
proj_volume = np.pi * (proj_diameter / 2) ** 2 * proj_length  # Volume in cubic meters
proj_mass = proj_density * proj_volume  # Mass in kg

SimulationParameters = namedtuple('SimulationParameters',
                                  ['init_volts',
                                   'capacitance',
                                   'resistance',
                                   'num_turns',
                                   'coil_diameter',
                                   'coil_length',
                                   'duration_s'])

# Define the differential equation for the RLC circuit
def sys_model(t, y, params):
    # Destructure the state vector
    [voltage, current, acceleration] = y

    # Rate of change in voltage
    dVdt = current / params.capacitance

    # Compute inductance, L
    coil_area = np.pi * (params.coil_diameter / 2) ** 2  # Coil cross-section, in m^2
    L = (params.num_turns ** 2 * mu_0 * coil_area) / params.coil_length  # Inductance in Henrys    

    # Rate of change in current 
    dIdt = -(params.resistance * current + voltage) / L

    # Compute force and acceleration on the projective
    mu_r = 5000  # Relative permeability of soft iron
    force = (1 / 2) * mu_0 ** 3 * mu_r * coil_area * params.num_turns ** 2 * current ** 2
    dvdt = force / proj_mass
    return [dVdt, dIdt, dvdt]

def run_sim(params):
    # Initial voltage across cap, init current across inductor, init velocity of projectile
    init_conditions = [-params.init_volts, 0, 0]
    t_span = (0, params.duration_s)  # Simulation time span
    t_eval = np.linspace(t_span[0], t_span[1], 10000)  # Set simulation time step
    # Solve the differential equation
    return solve_ivp(
        sys_model,
        t_span,
        init_conditions,
        t_eval=t_eval,
        args=(params,))

# Plot functions
def plot_circuit(solution):
    [voltage, current, acceleration] = solution.y
    time_sim = solution.t
    # Plot Current
    plt.figure(figsize=(8, 4))
    plt.plot(time_sim, current, 'r', label='Current (A)')
    plt.xlabel('Time (s)')
    plt.ylabel('Current (A)')
    plt.legend()
    plt.title('Current vs. Time')
    plt.grid(True)
    plt.tick_params(axis='y', labelcolor='r')
    plt.show()
    return solution

def plot_velocity_and_magnetism(time, vel, mag):
    fig, ax1 = plt.subplots(figsize=(8, 3))
    ax1.plot(time, vel, 'm', label='Velocity (m/s)')
    ax1.set_xlabel('Time (s)')  # Set x-axis label to seconds
    ax1.set_ylabel('Velocity (m/s)', color='m')
    ax1.tick_params(axis='y', labelcolor='m')
    ax2 = ax1.twinx()
    ax2.plot(time, mag, 'g', label='Magnetism (T)')
    ax2.set_xlabel('Time (s)')  # Set x-axis label to seconds
    ax2.set_ylabel('Magnetism (T)', color='g')
    ax2.tick_params(axis='y', labelcolor='g')
    plt.show()

def solve_and_plot(V0, C, R, N, D, l, duration_s):
    params = SimulationParameters(init_volts=V0,
                                     capacitance=C,
                                     resistance=R,
                                     num_turns=N,
                                     coil_diameter=D,
                                     coil_length=l,
                                     duration_s=duration_s)
    solution = run_sim(params)
    plots.plot_circuit(solution)


interact(
    solve_and_plot,
    V0=IntSlider(value=100, min=0, max=1000, step=5, description='V0 (Volts)'),
    C=FloatLogSlider(value=0.01, base=10, min=-6, max=2, step=0.1, description='C (Farads)'),
    R=FloatSlider(value=1, min=0.1, max=10, step=0.1, description='R'),
    N=FloatSlider(value=100, min=1, max=1000, description='Turns'),
    D=FloatSlider(value=.01, min=.001, max=.1, step=0.001, description='Diameter of coil (m)'),
    l=FloatSlider(value=.06, min=.01, max=.3, step=0.01, description='Length of coil (m)'),
    duration_s=FloatLogSlider(value=1, base=10, min=-3, max=2, step=0.1, description='Duration')
)
